# Part One - "Building Dataframe" #

Importing libraries and data from Wikipedia

In [1]:
import pandas as pd
import requests

In [2]:
#Requesting data from Wikipedia
www = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

#Converting data into Panadas dataframe
www_data = pd.read_html(www.content, header = 0)[0]
df = www_data.rename(columns={"Postal Code": "PostalCode"})
df = df.loc[df["Borough"] != "Not assigned"]
df.reset_index(inplace = True)
df = df[["PostalCode", "Borough", "Neighbourhood"]]
df.head()

PostalCode           Borough                                Neighbourhood
0        M3A        North York                                    Parkwoods
1        M4A        North York                             Victoria Village
2        M5A  Downtown Toronto                    Regent Park, Harbourfront
3        M6A        North York             Lawrence Manor, Lawrence Heights
4        M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government

In [3]:
df.groupby(['PostalCode']).first() #grouping dataframe by "PostalCode"

Borough                                      Neighbourhood
PostalCode                                                                
M1B         Scarborough                                     Malvern, Rouge
M1C         Scarborough             Rouge Hill, Port Union, Highland Creek
M1E         Scarborough                  Guildwood, Morningside, West Hill
M1G         Scarborough                                             Woburn
M1H         Scarborough                                          Cedarbrae
...                 ...                                                ...
M9N                York                                             Weston
M9P           Etobicoke                                          Westmount
M9R           Etobicoke  Kingsview Village, St. Phillips, Martin Grove ...
M9V           Etobicoke  South Steeles, Silverstone, Humbergate, Jamest...
M9W           Etobicoke                Northwest, West Humber - Clairville

[103 rows x 2 columns]

Checking if there are "Not assisgned" neighbourhoods

In [4]:
df.loc[df["Neighbourhood"] == "Not assigned"]

Empty DataFrame
Columns: [PostalCode, Borough, Neighbourhood]
Index: []

Printing the number of rows of your dataframe

In [5]:
df.shape

(103, 3)

# Part Two - "Adding coordinates to the neighbourhoods" #

Importing libraries and data into dataframe

In [6]:
#!pip install geocoder
import geocoder

In [7]:
url = 'http://cocl.us/Geospatial_data'
df_PCgeo = pd.read_csv(url)
df_PCgeo.head()

Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476

Comparing df and df_PCgeo before maeging data into one dataframe

In [8]:
df_PCgeo.shape

(103, 3)

Shape of df and df_PCgeo is the same. Additionally checking dtypes of df and df_PCgeo.

In [9]:
df.dtypes

PostalCode       object
Borough          object
Neighbourhood    object
dtype: object

In [10]:
df_PCgeo.dtypes #As it seems I need to change "Postal Code" to "PostalCode" before merging

Postal Code     object
Latitude       float64
Longitude      float64
dtype: object

In [11]:
df_PCgeo = df_PCgeo.rename(columns={"Postal Code": "PostalCode"})
df_PCgeo.dtypes

PostalCode     object
Latitude      float64
Longitude     float64
dtype: object

Marging df and df_PCgeo

In [12]:
df = df.join(df_PCgeo.set_index('PostalCode'), on='PostalCode')
df.head()

PostalCode           Borough                                Neighbourhood  \
0        M3A        North York                                    Parkwoods   
1        M4A        North York                             Victoria Village   
2        M5A  Downtown Toronto                    Regent Park, Harbourfront   
3        M6A        North York             Lawrence Manor, Lawrence Heights   
4        M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   

    Latitude  Longitude  
0  43.753259 -79.329656  
1  43.725882 -79.315572  
2  43.654260 -79.360636  
3  43.718518 -79.464763  
4  43.662301 -79.389494

# Part Three - "Exploring and clustering the neighborhoods in Toronto" #

Creating a map of Toronto with neighborhoods

In [13]:
import folium
#!pip install geopy
from geopy.geocoders import Nominatim

In [14]:
address = 'Toronto'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [15]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.Circle(
        [lat, lng],
        radius=500,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Defining Foursquare credentials and version

In [16]:
CLIENT_ID = 'FI5FBYT3V51GTQL3WVWKNKZ0PZIOSD2X3HR0J4LIXGWMS242'
CLIENT_SECRET = '2NKTS1RJPP514X05TMZ3LTBEGAZLNADKUTVGJ4NVB5V1STYG'
VERSION = '20180605'
LIMIT = 100
radius = 500

Exploring data for first neighbourhood in my dataframe to get sense of obtained data

In [17]:
n1_lat = df.loc[0, 'Latitude']
n2_long = df.loc[0, 'Longitude']
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    n1_lat, 
    n2_long, 
    radius, 
    LIMIT)
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fb56b2766505c5f6f45b90f'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 2,
  'suggestedBounds': {'ne': {'lat': 43.757758604500005,
    'lng': -79.32343823984928},
   'sw': {'lat': 43.7487585955, 'lng': -79.33587476015072}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'name': 'Brookbanks Park',
       'location': {'address': 'Toronto',
        'lat': 43.751976046055574,
        'lng': -79.33214044722958,
        'labeledLatLngs': 

Extracting relevant data and desiging functions for authomatization

In [18]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [19]:
import json
from pandas import json_normalize

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues)

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

name         categories        lat        lng
0  Brookbanks Park               Park  43.751976 -79.332140
1    Variety Store  Food & Drink Shop  43.751974 -79.333114

In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [21]:
venues_toronto = getNearbyVenues(names=df['Neighbourhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [22]:
venues_toronto.shape

(2166, 7)

In [23]:
venues_toronto.head(10)

Neighbourhood  Neighbourhood Latitude  Neighbourhood Longitude  \
0                  Parkwoods               43.753259               -79.329656   
1                  Parkwoods               43.753259               -79.329656   
2           Victoria Village               43.725882               -79.315572   
3           Victoria Village               43.725882               -79.315572   
4           Victoria Village               43.725882               -79.315572   
5           Victoria Village               43.725882               -79.315572   
6           Victoria Village               43.725882               -79.315572   
7           Victoria Village               43.725882               -79.315572   
8  Regent Park, Harbourfront               43.654260               -79.360636   
9  Regent Park, Harbourfront               43.654260               -79.360636   

                                       Venue  Venue Latitude  Venue Longitude  \
0                            Brookbanks Park       43.751976       -79.332140   
1                              Variety Store       43.751974       -79.333114   
2                     Victoria Village Arena       43.723481       -79.315635   
3                                  Portugril       43.725819       -79.312785   
4                                Tim Hortons       43.725517       -79.313103   
5                                   The Frig       43.727051       -79.317418   
6  Eglinton Ave E & Sloane Ave/Bermondsey Rd       43.726086       -79.313620   
7                                 Pizza Nova       43.725824       -79.312860   
8                           Roselle Desserts       43.653447       -79.362017   
9                              Tandem Coffee       43.653559       -79.361809   

          Venue Category  
0                   Park  
1      Food & Drink Shop  
2           Hockey Arena  
3  Portuguese Restaurant  
4            Coffee Shop  
5      French Restaurant  
6           Intersection  
7            Pizza Place  
8                 Bakery  
9            Coffee Shop

In [24]:
venues_toronto.groupby('Neighbourhood').count()

Neighbourhood Latitude  \
Neighbourhood                                                             
Agincourt                                                             5   
Alderwood, Long Branch                                                7   
Bathurst Manor, Wilson Heights, Downsview North                      21   
Bayview Village                                                       4   
Bedford Park, Lawrence Manor East                                    22   
...                                                                 ...   
Willowdale, Willowdale West                                           5   
Woburn                                                                4   
Woodbine Heights                                                      9   
York Mills West                                                       2   
York Mills, Silver Hills                                              1   

                                                 Neighbourhood Longitude  \
Neighbourhood                                                              
Agincourt                                                              5   
Alderwood, Long Branch                                                 7   
Bathurst Manor, Wilson Heights, Downsview North                       21   
Bayview Village                                                        4   
Bedford Park, Lawrence Manor East                                     22   
...                                                                  ...   
Willowdale, Willowdale West                                            5   
Woburn                                                                 4   
Woodbine Heights                                                       9   
York Mills West                                                        2   
York Mills, Silver Hills                                               1   

                                                 Venue  Venue Latitude  \
Neighbourhood                                                            
Agincourt                                            5               5   
Alderwood, Long Branch                               7               7   
Bathurst Manor, Wilson Heights, Downsview North     21              21   
Bayview Village                                      4               4   
Bedford Park, Lawrence Manor East                   22              22   
...                                                ...             ...   
Willowdale, Willowdale West                          5               5   
Woburn                                               4               4   
Woodbine Heights                                     9               9   
York Mills West                                      2               2   
York Mills, Silver Hills                             1               1   

                                                 Venue Longitude  \
Neighbourhood                                                      
Agincourt                                                      5   
Alderwood, Long Branch                                         7   
Bathurst Manor, Wilson Heights, Downsview North               21   
Bayview Village                                                4   
Bedford Park, Lawrence Manor East                             22   
...                                                          ...   
Willowdale, Willowdale West                                    5   
Woburn                                                         4   
Woodbine Heights                                               9   
York Mills West                                                2   
York Mills, Silver Hills                                       1   

                                                 Venue Category  
Neighbourhood                                                    
Agincourt                                                     5  
Alderwood, Long Branch                

Having all relevant data in the dataframe called "venues_toronto", I am going to produce top-10 categories for each neighbourhood now.

In [25]:
toronto_onehot = pd.get_dummies(venues_toronto[['Venue Category']], prefix="", prefix_sep="")

toronto_onehot['Neighbourhood'] = venues_toronto['Neighbourhood'] 

fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

Neighbourhood  Accessories Store  Afghan Restaurant  Airport  \
0         Parkwoods                  0                  0        0   
1         Parkwoods                  0                  0        0   
2  Victoria Village                  0                  0        0   
3  Victoria Village                  0                  0        0   
4  Victoria Village                  0                  0        0   

   Airport Food Court  Airport Gate  Airport Lounge  Airport Service  \
0                   0             0               0                0   
1                   0             0               0                0   
2                   0             0               0                0   
3                   0             0               0                0   
4                   0             0               0                0   

   Airport Terminal  American Restaurant  ...  Train Station  \
0                 0                    0  ...              0   
1                 0                    0  ...              0   
2                 0                    0  ...              0   
3                 0                    0  ...              0   
4                 0                    0  ...              0   

   Turkish Restaurant  Vegetarian / Vegan Restaurant  Video Game Store  \
0                   0                              0                 0   
1                   0                              0                 0   
2                   0                              0                 0   
3                   0                              0                 0   
4                   0                              0                 0   

   Vietnamese Restaurant  Warehouse Store  Wine Bar  Wings Joint  \
0                      0                0         0            0   
1                      0                0         0            0   
2                      0                0         0            0   
3                      0                0         0            0   
4                      0                0         0            0   

   Women's Store  Yoga Studio  
0              0            0  
1              0            0  
2              0            0  
3              0            0  
4              0            0  

[5 rows x 278 columns]

In [26]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.loc[toronto_grouped['Korean Restaurant'] > 0]

Neighbourhood  Accessories Store  Afghan Restaurant  Airport  \
13        Central Bay Street                0.0                0.0      0.0   
48  Little Portugal, Trinity                0.0                0.0      0.0   

    Airport Food Court  Airport Gate  Airport Lounge  Airport Service  \
13                 0.0           0.0             0.0              0.0   
48                 0.0           0.0             0.0              0.0   

    Airport Terminal  American Restaurant  ...  Train Station  \
13               0.0                  0.0  ...            0.0   
48               0.0                  0.0  ...            0.0   

    Turkish Restaurant  Vegetarian / Vegan Restaurant  Video Game Store  \
13                 0.0                       0.014706               0.0   
48                 0.0                       0.021739               0.0   

    Vietnamese Restaurant  Warehouse Store  Wine Bar  Wings Joint  \
13               0.000000              0.0  0.014706          0.0   
48               0.043478              0.0  0.021739          0.0   

    Women's Store  Yoga Studio  
13            0.0     0.014706  
48            0.0     0.021739  

[2 rows x 278 columns]

In [27]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [28]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

Neighbourhood 1st Most Common Venue  \
0                                        Agincourt                Lounge   
1                           Alderwood, Long Branch           Pizza Place   
2  Bathurst Manor, Wilson Heights, Downsview North           Coffee Shop   
3                                  Bayview Village   Japanese Restaurant   
4                Bedford Park, Lawrence Manor East        Sandwich Place   

  2nd Most Common Venue      3rd Most Common Venue 4th Most Common Venue  \
0        Breakfast Spot  Latin American Restaurant          Skating Rink   
1        Sandwich Place                Coffee Shop                   Pub   
2                  Bank                   Pharmacy        Ice Cream Shop   
3                  Café         Chinese Restaurant                  Bank   
4    Italian Restaurant                Coffee Shop      Greek Restaurant   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0        Clothing Store           Event Space    Falafel Restaurant   
1              Pharmacy                   Gym      Greek Restaurant   
2         Shopping Mall           Bridal Shop        Sandwich Place   
3   Distribution Center              Dive Bar               Dog Run   
4       Thai Restaurant             Locksmith          Liquor Store   

     8th Most Common Venue      9th Most Common Venue 10th Most Common Venue  
0     Ethiopian Restaurant                Escape Room    Distribution Center  
1           Discount Store           Department Store           Dessert Shop  
2                    Diner  Middle Eastern Restaurant             Restaurant  
3         Doner Restaurant                 Donut Shop            Yoga Studio  
4  Comfort Food Restaurant                  Juice Bar                Butcher

Time to cluster neighbourhoods (k-means)

In [29]:
from sklearn.cluster import KMeans

kclusters = 10

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

kmeans.labels_[0:10]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [30]:
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

df_neighANDvenues = df
df_neighANDvenues = df_neighANDvenues.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

df_neighANDvenues.head()

PostalCode           Borough                                Neighbourhood  \
0        M3A        North York                                    Parkwoods   
1        M4A        North York                             Victoria Village   
2        M5A  Downtown Toronto                    Regent Park, Harbourfront   
3        M6A        North York             Lawrence Manor, Lawrence Heights   
4        M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   

    Latitude  Longitude  Cluster Labels 1st Most Common Venue  \
0  43.753259 -79.329656             0.0                  Park   
1  43.725882 -79.315572             1.0           Pizza Place   
2  43.654260 -79.360636             1.0           Coffee Shop   
3  43.718518 -79.464763             1.0        Clothing Store   
4  43.662301 -79.389494             1.0           Coffee Shop   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0     Food & Drink Shop           Yoga Studio   Dumpling Restaurant   
1          Hockey Arena     French Restaurant           Coffee Shop   
2                  Park                Bakery                   Pub   
3     Accessories Store              Boutique             Gift Shop   
4           Yoga Studio                  Bank              Beer Bar   

    5th Most Common Venue 6th Most Common Venue       7th Most Common Venue  \
0     Distribution Center              Dive Bar                     Dog Run   
1   Portuguese Restaurant          Intersection  Construction & Landscaping   
2          Breakfast Spot                  Café                     Theater   
3  Furniture / Home Store           Event Space                 Coffee Shop   
4           Smoothie Shop        Sandwich Place                        Café   

  8th Most Common Venue  9th Most Common Venue 10th Most Common Venue  
0      Doner Restaurant             Donut Shop              Drugstore  
1                 Diner   Ethiopian Restaurant            Escape Room  
2                   Spa                Brewery             Shoe Store  
3         Women's Store  Vietnamese Restaurant        Airport Service  
4            Restaurant     Chinese Restaurant  Portuguese Restaurant

Visualizing on the map

In [31]:
import matplotlib.cm as cm
import matplotlib.colors as colors

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

df_neighANDvenues_nonan = df_neighANDvenues.dropna(subset=['Cluster Labels'])

markers_colors = []
for lat, lon, poi, cluster in zip(df_neighANDvenues_nonan['Latitude'], df_neighANDvenues_nonan['Longitude'], df_neighANDvenues_nonan['Neighbourhood'], df_neighANDvenues_nonan['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# The end #